In [ ]:
import os
import cv2
import glob
import PIL
import shutil
import numpy as np
import pandas as pd
from architectures.Unet3D import *
from metrics3D import *
from datagenerators import *

from tensorflow import keras
import keras.backend as K
from keras.callbacks import CSVLogger
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers.experimental import preprocessing

np.set_printoptions(precision=3, suppress=True)

CSVLogger.on_test_begin = CSVLogger.on_train_begin
CSVLogger.on_test_batch_end = CSVLogger.on_epoch_end
CSVLogger.on_test_end = CSVLogger.on_train_end

In [ ]:
model = build_3Dunet(n_channels=3, ker_init='he_normal', dropout=0.2)
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.01), metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=4), dice_coef, dice_coef_necrotic, dice_coef_edema ,dice_coef_enhancing] )

In [ ]:
directory = [f.path for f in os.scandir(PATH) if f.is_dir()]


def generate_ids(dirLst):
    ids = []
    for i in range(0,len(dirLst)):
        ids.append(dirLst[i][dirLst[i].rfind('/')+1:])
    return ids

train_and_test_ids = generate_ids(directory); 

    
train_test_ids, val_ids = train_test_split(train_and_test_ids, train_size=236, test_size=60, random_state=14) 
train_ids, test_ids = train_test_split(train_test_ids, test_size=36, random_state=14) 

In [ ]:
print(len(train_ids))
print(len(val_ids))
print(len(test_ids))

In [ ]:
training_generator = DataGenerator3D(train_ids)
valid_generator = DataGenerator3D(val_ids)
test_generator = DataGenerator3D(test_ids)

In [ ]:
csv_logger = CSVLogger('Logs/training/3D_U_net_instancenormnodrop.log', separator=',', append=True)

eval_csv_logger = CSVLogger('Logs/testing/3D_U_net_instancenormnodrop.log', separator=',', append=True)

callbacks = [
      keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0000001, verbose=1),
        csv_logger
    ]

In [ ]:
K.clear_session()

history =  model.fit(training_generator,
                    epochs=100,
                    steps_per_epoch=len(train_ids),
                    callbacks= callbacks,
                    validation_data = valid_generator
                    )  
model.save("Models/3D/3D_U_net_instancenormfulldrop.h5")

In [ ]:
model = keras.models.load_model('Models/3D/3D_U_net_instancenormnodrop.h5', 
                                   custom_objects={ 'accuracy' : tf.keras.metrics.MeanIoU(num_classes=4),
                                                   "dice_coef": dice_coef,
                                                   "dice_coef_necrotic": dice_coef_necrotic,
                                                   "dice_coef_edema": dice_coef_edema,
                                                   "dice_coef_enhancing": dice_coef_enhancing
                                                  }, compile=True)

In [ ]:
model.evaluate(test_generator, callbacks=eval_csv_logger, verbose=1)

In [ ]:
valid_csv_logger = CSVLogger('Logs/validation/3D_U_net_instancenormfulldrop.log', separator=',', append=True)
model.evaluate(valid_generator, callbacks=valid_csv_logger, verbose=1)